# greedy

In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("../Dataset/ShipCase1.csv")
df2 = pd.read_csv("../Dataset/ShipCase2.csv")
df3 = pd.read_csv("../Dataset/ShipCase3.csv")
df4 = pd.read_csv("../Dataset/ShipCase4.csv")
df5 = pd.read_csv("../Dataset/ShipCase5.csv")
df6 = pd.read_csv("../Dataset/ShipCase6.csv")

1. Define the representation and initial state

*Note that crane located at [0, 8] (by slides, just above [1, 8])*

In [3]:
# Store mainfest: NAN as -1, 0 as empty, >0 as occupied, number are weight of the container
def create_grid(path):
    df = pd.read_csv(path)
    nrows = int(df["row"].max()) + 1
    ncols = int(df["column"].max())
    grid = np.zeros((nrows, ncols), dtype=int)

    for _, rec in df.iterrows():
        r = nrows - int(rec["row"])
        c = int(rec["column"]) - 1
        desc = str(rec["description"]).strip().upper()

        if desc == "NAN":
            grid[r, c] = -1
        elif desc == "UNUSED":
            grid[r, c] = 0
        else:
            grid[r, c] = int(rec["weight"])
    return grid

grid6 = create_grid("../Dataset/ShipCase6.csv")
grid6

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ -1,  61,  40, 334, 333, 333,   0,   0,   0,   0, 900,  -1]])

2. Define goal state

In [4]:
def left_weight(grid):
    return int(np.clip(grid[1:9, 0:6], 0, None).sum())

def right_weight(grid):
    return int(np.clip(grid[1:9, 6:12], 0, None).sum())

def goal_test(grid, tol=0.10):
    total = left_weight(grid) + right_weight(grid)
    return True if total == 0 else abs(left_weight(grid) - right_weight(grid)) <= tol * total

goal_test(grid6)

False

3. Define the operators (What to move)

In [5]:
# what to move: determine which container to move
def determine_container_to_move(grid):
    # compute which side is heavier
    left_sum = left_weight(grid)
    right_sum = right_weight(grid)

    low, high = (0, 6) if left_sum > right_sum else (6, 12)
    
    # comopute the needed weight to move
    total = left_sum + right_sum
    diff = abs(left_sum - right_sum)
    target = total * 0.1
    needed = math.ceil((diff - target) / 2)
    
    # find the container to move
    candidates = [((row, col), int(grid[row, col]))
                    for row in range(grid.shape[0])
                    for col in range(low, high)
                    if int(grid[row, col]) > 0]
    
    # a. check if can do done by move single container
    for pos, weight in candidates:
        if weight >= needed:
            return [(pos, weight)], needed
        # else: continue
    
    # b. otherwise, try combinations containers
    sorted_candidates = sorted(candidates, key=lambda x: x[1])
    current_combination = []
    current_weight = 0

    for pos, weight in sorted_candidates:
        current_combination.append((pos, weight))
        current_weight += weight
        if current_weight >= needed:
            return current_combination, needed

    return candidates, needed

candidates, needed = determine_container_to_move(grid6)
candidates, needed


([((8, 1), 61)], 1)

After determine which container need to move, now it's time to move them (greedy move):

4. Search algorithm (Actual Operation)

    - Check if any container above it. If yes, clear it by moving all above container to nearest columns on the same side. 
    - Slide horizontally to the right: grid to the right might be blocked, move up until right cell is empty; then move right; repeat if same silution happen. 
    - Drop down. Once on the target side, drop it down. To save steps, can drop on columns that nearest the border. 

In [6]:
# 1. check if any container above
def has_container_above(grid, position):
    row, col = position
    for r in range(0, row):
        if grid[r, col] > 0:
            return True
    return False

# 2. if yes, clear it by moving all above container to nearest columns on the same side
def clear_above_containers(grid, position):
    # get target position
    row, col = position
    moves = []
    
    # choose side column range
    if 0<= col < 6:
        col_range = range(0, 6)   # left side
    else:
        col_range = range(6, 12)  # right side
        
    # start from topmost container above target, then move downwards
    for r in range(0, row):
        if grid[r, col] <= 0:
            continue
        
        # find nearest empty column on the same side
        for c in col_range:
            if grid[r, c] == 0:
                moves.append(((r, col), (r, c)))
                grid[r, c] = grid[r, col]
                grid[r, col] = 0
                break

In [7]:
# check this on main
# if (!goal_test(grid)):
#     raise ValueError("The initial grid does not meet the balance goal.")